# Loading and modifying layers and layer collections

In [1]:
import geoengine as ge
from datetime import datetime

/home/droenner/git/geoengine-python/env/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
ge.initialize("http://localhost:3030/api")

## Retrieve layer collections

In [3]:
root_collection = ge.layer_collection()
root_collection

Layer Collection
name: Layer Providers
description: All available Geo Engine layer providers
id: f2424474-ef24-4c18-ab84-68592e12ce48
provider id: 1c3b8042-300b-485c-95b5-0147d9dc068d
items: Layer Collection
name: Datasets
description: Basic Layers for all Datasets
id: 546073b6-d535-4205-b601-99675c9f6dd7
provider id: ac50ed0d-c9a0-41f8-9ce8-35fc9e38299b

       Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: MockName
description: MockType
id: b5f82c7c-9133-4ac1-b4ae-8faac3b9a6df
provider id: d0535f1d-27b6-4982-b2f8-b1070f1bf6ee

In [4]:
root_of_layerdb = root_collection.items[1].load()
root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [5]:
test_collection = root_of_layerdb.items[0].load()
test_collection

Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [6]:
layer = test_collection.items[1].load()
layer

Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
workflow: {'type': 'Vector', 'operator': {'type': 'PointInPolygonFilter', 'params': {}, 'sources': {'points': {'type': 'OgrSource', 'params': {'data': {'type': 'internal', 'datasetId': 'a9623a5b-b6c5-404b-bc5a-313ff72e4e75'}, 'attributeProjection': None, 'attributeFilters': None}}, 'polygons': {'type': 'OgrSource', 'params': {'data': {'type': 'internal', 'datasetId': 'b6191257-6d61-4c6b-90a4-ebfb1b23899d'}, 'attributeProjection': None, 'attributeFilters': None}}}}}
symbology: None
properties: []
metadata: {}

In [7]:
workflow = layer.as_workflow()

time = datetime.strptime('2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")

data = workflow.get_dataframe(
    ge.QueryRectangle(
        ge.BoundingBox2D(-111.533203125, -4.482421875, 114.345703125, 73.388671875),
        ge.TimeInterval(time, time),
        ge.SpatialResolution(0.1, 0.1)
        )
    )

data

,geometry,featurecla,name,natlscale,scalerank,website,start,end
0,POINT (8.70724 53.86472),Port,Cuxhaven,5.0,8,www.cuxport.de,NaT,NaT
1,POINT (9.83581 54.47361),Port,Eckernforde,5.0,8,www.stadtwerke-eckernfoerde.de,NaT,NaT
2,POINT (8.48952 53.48750),Port,Nordenham,5.0,8,None,NaT,NaT
3,POINT (7.36896 53.09639),Port,Papenburg,5.0,8,None,NaT,NaT
4,POINT (11.43916 53.90639),Port,Wismar,5.0,8,www.hafen-wismar.de,NaT,NaT
5,POINT (10.86826 53.95222),Port,Travemunde,10.0,7,None,NaT,NaT
6,POINT (7.19287 53.34611),Port,Emden,20.0,6,www.seaport-emden.de,NaT,NaT
7,POINT (9.43687 54.80333),Port,Flensburg,20.0,6,www.flensburg-tourismus.de,NaT,NaT
8,POINT (8.12214 53.53000),Port,Wilhelmshaven,20.0,6,www.wilhelmshaven-port.de,NaT,NaT
9,POINT (8.75106 53.09750),Port,Bremen,30.0,5,www.bremen-ports.de,NaT,NaT


## Add collections and layers

In [8]:
root_of_layerdb.add_collection("my test collection", "test description")
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: my test collection
description: test description
id: 2b409186-f861-4fae-8132-42abadcafc4a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [9]:
test_collection = root_collection.get_items_by_name('my test collection')[0].load()
test_collection

IndexError: list index out of range

In [ ]:
test_collection.add_collection("sub collection", "another description")

test_collection.add_layer(
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": {
                            "type": "internal",
                            "datasetId": "a9623a5b-b6c5-404b-bc5a-313ff72e4e75"
                        },
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": {
                            "type": "internal",
                            "datasetId": "b6191257-6d61-4c6b-90a4-ebfb1b23899d"
                        },
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

test_collection = test_collection.reload()
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 22be7fac-be04-4e77-8e44-37a5591e3d45
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
sub_collection = test_collection.items[0].load()
sub_collection.add_existing_layer(test_collection.items[1])

sub_collection.add_collection("sub sub collection", "yet another description")

sub_collection = sub_collection.reload()
sub_collection

Layer Collection
name: sub collection
description: another description
id: 22be7fac-be04-4e77-8e44-37a5591e3d45
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub sub collection
description: yet another description
id: a1c8654d-423c-4f59-8ec2-f6221696e9c2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection.add_existing_collection(sub_collection.items[0])

test_collection = test_collection.reload()
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 22be7fac-be04-4e77-8e44-37a5591e3d45
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: sub sub collection
description: yet another description
id: a1c8654d-423c-4f59-8ec2-f6221696e9c2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Delete previously added collections and layers

In [ ]:
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 22be7fac-be04-4e77-8e44-37a5591e3d45
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: sub sub collection
description: yet another description
id: a1c8654d-423c-4f59-8ec2-f6221696e9c2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub sub collection
description: yet another description
id: a1c8654d-423c-4f59-8ec2-f6221696e9c2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer
name: ports clone
description: test description
id: fcc0d133-6d5e-4244-86cf-0be29665d61a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
root_of_layerdb.reload()

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: my test collection
description: test description
id: c9c12672-880a-450f-8d6f-d2feafb77de2
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection.remove()

In [ ]:
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74